# CLSS Template Import Notebook

This notebook provides a guided workflow for importing CLSS templates and related configuration data into ArcGIS Online feature services. Use this process to update or initialize your CLSS feature services with the latest templates, ensuring your lifeline assessments and configurations are current and standardized.

## What does this notebook do?

- **Imports CLSS templates** from CSV files exported by the companion 'template-export' notebook.
- **Maps and uploads template data** to a selected ArcGIS Online feature service table.
- **Automates the creation of structured lifeline/component/indicator JSON** for use in CLSS workflows.

---

## Prerequisites

Before running this notebook, please ensure:

1. **Template Export:**  
   Export templates from CSV using the Export Template process (see user guide). Import templates to a working directory (e.g., `/home/export` folder).


2. **Feature Service Access:**  
   You have access to the target CLSS ArcGIS Online feature service and know its item ID.  You will also need to run this notebook with sufficient priviliges to add/update featur service data. 

---

## Steps

1. **Select Feature Service:**  
   Set the `selected_fs_id` variable to the item ID of your target CLSS feature service.

2. **Set CSV File Path:**  
   Update the `csv_file_path` variable to point to your exported template CSV.

3. **Run the Notebook:**  
   The notebook will authenticate, load your template data, generate the required JSON, and upload it to the feature service.

---


#### Run this cell to connect to your GIS and get started:

In [25]:
from arcgis.gis import GIS
import pandas as pd
from io import BytesIO
import json

gis = GIS("home")

You are logged on as cusec_hub with an administrator role, proceed with caution.


In [26]:
# ✅ USER: Set your target FS and CSV file path here
selected_fs_id = '99e2deda90c04ca7a7ab95f09113783c'  # add the target CLSS Feature Service ID here
csv_file_path = '/arcgis/home/templates/template_name.csv'  # <- Update to your local path and template csv file


#### Now you are ready to start!

In [27]:
# Initialize GIS (make sure `gis` is defined earlier or authenticate here)
fs_item = gis.content.get(selected_fs_id)
target_template_table = fs_item.tables[9]  # assumes sublayer 12, which is in position 9 in the feature service


In [30]:
# Load the CSV
try:
    template_indicator_df = pd.read_csv(csv_file_path)
    print(f"✅ Loaded CSV with {len(template_indicator_df)} indicator rows")
except Exception as e:
    raise Exception(f"❌ Error reading CSV: {e}")

# Function to generate structured nested indicator JSON
def return_indicator_json(template_indicator_df):
    template_json = []
    for lifeline, lifeline_group in template_indicator_df.groupby('Lifeline'):
        lifeline_dict = {
            "name": lifeline,
            "title": lifeline,
            "componentTemplates": []
        }
        for component, component_group in lifeline_group.groupby('Component'):
            component_dict = {
                "name": component,
                "title": component,
                "indicators": []
            }
            for _, row in component_group.iterrows():
                indicator_dict = {
                    "name": row['Indicator'],
                    "templateName": row['TemplateName'],
                    "lifelineName": lifeline,
                    "componentName": component,
                    "weight": row['Weight']
                }
                component_dict["indicators"].append(indicator_dict)
            lifeline_dict["componentTemplates"].append(component_dict)
        template_json.append(lifeline_dict)
    return template_json

# Function to add template to FS
def add_to_feature_service(template_info_df, template_json):
    # Get the first template metadata record
    row = template_info_df.iloc[0]

    name = row['TemplateName']
    description = row['TemplateDescription']
    status = row.get('Status', 1)  # Default to 1 (Published) if missing
    isDeleted = 0  # Default to 0 (not deleted)
    
    try:
        status = int(status)
    except:
        status = 1

    # Construct one feature with full JSON content
    feature = {
        "attributes": {
            "Name": name,
            "Description": description,
            "Status": status,
            "IsDeleted": isDeleted,
            "Content": json.dumps(template_json, ensure_ascii=True)
            # "Content": json.dumps(template_json, ensure_ascii=True).replace('"', '\\"')
        }
    }

    try:
        response = target_template_table.edit_features(adds=[feature])

        # Check for success
        if not response.get("addResults", [{}])[0].get("success", False):
            error = response['addResults'][0].get("error", {}).get("description", "Unknown error")
            print(f"❌ Upload failed: {error}")
        else:
            print(f"✅ Successfully added 1 template to: {target_feature_service.title}")

    except Exception as e:
        print(f"❌ Exception during upload: {e}")


# Run the import
template_json = return_indicator_json(template_indicator_df)
add_to_feature_service(template_indicator_df, template_json)

✅ Loaded CSV with 215 indicator rows
✅ Successfully added 1 template to: CLSS_FeatureService_sandbox
